In [122]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import os
import scipy.signal

from spykshrk.realtime.decoder_process import PointProcessDecoder

import spykshrk.realtime.simulator.nspike_data as nspike_data

from spykshrk.franklab.pp_decoder.pp_clusterless import plot_decode_2d

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas

from spykshrk.franklab.pp_decoder.decode_error import bin_pos_data, calc_error_table, \
                                                      plot_arms_error

from spykshrk.franklab.pp_decoder.data_containers import LinearPositionContainer
    
#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 180)

 
idx = pd.IndexSlice


In [8]:
%%time
# Load merged rec HDF store based on config

config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv_300samp/bond.config.json'
config = json.load(open(config_file, 'r'))

hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

store = pd.HDFStore(hdf_file, mode='r')
spike_decode = store['rec_3']
decoder_df = store['rec_4']
stim_lockout = store['rec_11']

In [9]:
%%time
stim_lockout_ranges = stim_lockout.pivot(index='lockout_num',columns='lockout_state', values='timestamp')
stim_lockout_ranges = stim_lockout_ranges.reindex(columns=[1,0])


In [10]:
%%time
# Get table with decode for each spike and generate decode bin mask

dec_bin_size = 300     # Decode bin size in samples (usually 30kHz)

arm_coordinates = [[0, 69], [150, 150+102], [300, 300+104]]

start_timestamp = spike_decode['timestamp'][0]
dec_bins = np.floor((spike_decode['timestamp'] - spike_decode['timestamp'][0])/dec_bin_size).astype('int')
dec_bins_start = int(spike_decode['timestamp'][0] / dec_bin_size) * dec_bin_size + dec_bins * dec_bin_size
spike_decode['dec_bin'] = dec_bins
spike_decode['dec_bin_start'] = dec_bins_start


pos_upper = config['encoder']['position']['upper']
pos_lower = config['encoder']['position']['lower']
pos_num_bins = config['encoder']['position']['bins']
pos_bin_delta = ((pos_upper - pos_lower) / pos_num_bins)

x_bins = np.linspace(0, pos_bin_delta*(pos_num_bins-1), pos_num_bins)
x_bin_edges = np.linspace(0, pos_bin_delta*(pos_num_bins), pos_num_bins+1)

pos_kernel = gaussian(x_bins, x_bins[int(len(x_bins)/2)], 3)


In [118]:
# Get real position

nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

# Transform position into simpler table with only linear position
pos_data_linpos = simplify_pos_pandas(pos_data)

# Convert real pos to realtime system linear map (single linear coordinate)

center_pos_flat = pos_data_linpos[pos_data_linpos['seg_idx'] == 1]['well_center']
left_pos_flat = pos_data_linpos[(pos_data_linpos['seg_idx'] == 2) | 
                                (pos_data_linpos['seg_idx'] == 3)]['well_left'] + 150
right_pos_flat = pos_data_linpos[(pos_data_linpos['seg_idx'] == 4) | 
                                 (pos_data_linpos['seg_idx'] == 5)]['well_right'] + 300

center_pos_flat.name = 'linpos_flat'
left_pos_flat.name = 'linpos_flat'
right_pos_flat.name = 'linpos_flat'

linpos_flat = pd.concat([center_pos_flat, left_pos_flat, right_pos_flat])
linpos_flat = linpos_flat.sort_index()

In [175]:
lin_con = LinearPositionContainer(pos_data)
lin_con.get_rebinned(60)

In [119]:
pos_data.index.set_levels(

In [120]:
pos_data.reindex([(4,1,73830339), (4,1,73830340)], method='bfill')

In [149]:
from itertools import product
test = pd.DataFrame(np.ones([6,2]), index=pd.MultiIndex.from_tuples([(1,1,1),(1,1,2),(1,1,3),(1,2,1),(1,2,2),(2,1,1)]))
test


In [155]:
grp = test.groupby(level=[0,1])
grp.apply(lambda x: print(np.unique(x.index.get_level_values(0))))

    

In [139]:
test.append([(2,1,1), [1,1]])

In [14]:
%%time
# Run spykshrk.realtime version of point process decoding

pp_decoder = PointProcessDecoder(pos_range=[config['encoder']['position']['lower'],
                                            config['encoder']['position']['upper']],
                                 pos_bins=config['encoder']['position']['bins'],
                                 time_bin_size=config['pp_decoder']['bin_size'],
                                 arm_coor=config['pp_decoder']['arm_pos'],
                                 uniform_gain=config['pp_decoder']['trans_mat_uniform_gain'])

pp_decoder.select_ntrodes(config['simulator']['nspike_animal_info']['tetrodes'])

num_time_bins = spike_decode.loc[:,'dec_bin'].max()

groups = spike_decode.groupby('dec_bin')

last_bin_id = 0

spykshrk_posteriors = np.zeros([num_time_bins+1, pos_num_bins])

dec_bin_times = []

for bin_id, spikes_in_bin in groups:
    if last_bin_id <= bin_id - 1:
        for bin_no_spk_id in range(last_bin_id + 1, bin_id):
            # No spike in bin
            
            dec_bin_times.append(start_timestamp + bin_no_spk_id * dec_bin_size)
            post = pp_decoder.increment_no_spike_bin()
            spykshrk_posteriors[bin_no_spk_id, :] = post
        
    for ntrode_id, dec in zip(spikes_in_bin.loc[:, 'ntrode_id'].values, 
                              spikes_in_bin.loc[:, 'x0': 'x{:d}'.format(pos_num_bins-1)].values):
        pp_decoder.add_observation(ntrode_id, dec)
        
    post = pp_decoder.increment_bin()
    dec_bin_times.append(spikes_in_bin['dec_bin_start'].values[0])
    spykshrk_posteriors[bin_id, :] = post
    last_bin_id = bin_id
    

In [ ]:
# plt_ranges = [[2461, 3404]]
plt_ranges = [[2461 + 700, 2461+900]]

for plt_range in plt_ranges:
    plt.figure(figsize=(400,20))
    plt.subplot(2,1,1)
    plot_decode_2d(decoder_df['timestamp'], 
                   decoder_df.loc[:,'x0':'x449'].values,
                   stim_lockout_ranges, 
                   decoder_df.set_index('real_pos_time')['real_pos'], plt_range, 1.0)
    
    plt.subplot(2,1,2)
    plot_decode_2d(np.array(dec_bin_times),
                   spykshrk_posteriors,
                   stim_lockout_ranges, 
                   linpos_flat, plt_range, 1.0)

    
plt.show()

In [69]:
dec_est_map = x_bins[np.argmax(decoder_df.loc[:,'x0':'x449'].values, axis=1)]
dec_est_pos = pd.DataFrame({'est_pos': dec_est_map}, index=pd.Index(data=decoder_df['timestamp'],
                                                                    name='timestamp'))

pos_data_bins = bin_pos_data(pos_data_linpos, dec_bin_size)

center_dec_error, left_dec_error, right_dec_error = calc_error_table(pos_data_bins, dec_est_pos,
                                                                     arm_coordinates, 2)

print('median error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.median(center_dec_error['abs_error']),
                                                                       np.median(left_dec_error['abs_error']),
                                                                       np.median(right_dec_error['abs_error'])))

print('mean error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.mean(center_dec_error['abs_error']),
                                                                       np.mean(left_dec_error['abs_error']),
                                                                       np.mean(right_dec_error['abs_error'])))

In [70]:
dec_est_map = x_bins[np.argmax(spykshrk_posteriors, axis=1)]
dec_est_pos = pd.DataFrame({'est_pos': dec_est_map}, index=pd.Index(data=np.array(dec_bin_times),
                                                                    name='timestamp'))

pos_data_bins = bin_pos_data(pos_data_linpos, dec_bin_size)

center_dec_error, left_dec_error, right_dec_error = calc_error_table(pos_data_bins, dec_est_pos,
                                                                     arm_coordinates, 2)

print('median error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.median(center_dec_error['abs_error']),
                                                                       np.median(left_dec_error['abs_error']),
                                                                       np.median(right_dec_error['abs_error'])))

print('mean error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.mean(center_dec_error['abs_error']),
                                                                       np.mean(left_dec_error['abs_error']),
                                                                       np.mean(right_dec_error['abs_error'])))